<a href="https://colab.research.google.com/github/buganart/BUGAN/blob/master/latent_space_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [ ]:
#@markdown Mount google drive.
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

# Check if we have linked the folder
from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    print(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

In [ ]:
#@title Configure dataset
#@markdown - choose whether to use wandb id or use checkpoint_path to select checkpoint file

#@markdown Choice 1: wandb id and project_name to select checkpoint file
#@markdown - set `"run_id"` to resume a run (for example: `u9imsvva`)
#@markdown - The id of the current run is shown below in the cell with `wandb.init()` or you can find it in the wandb web UI.
id = "2hg5kfx4" #@param {type:"string"}
#@markdown - Enter project name (either `handtool-gan` or `tree-gan`)
project_name = "tree-gan" #@param ["tree-gan", "handtool-gan"]

#@markdown Choice 2: file path and model type to select saved checkpoint .ckpt file
#@markdown - For example via the file browser on the left to locate and right click to copy the path.
#@markdown - file path example: `/content/drive/My Drive/h/k/checkpoint.ckpt` 
ckpt_file_location = "" #@param {type:"string"}
#@markdown - Enter trained neural network model type
#@markdown - (may be necessary for wandb_id if selected_model is not saved in config)
selected_model = "VAE"    #@param ["VAEGAN", "GAN", "VAE"]

#@markdown For VAEGAN/VAE only: Enter 2 mesh file location to specify 2 latent vectors.   
meshfile1 = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj/obj_files/old_1.obj" #@param {type:"string"}
meshfile2 = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Research/Peter/Tree_3D_models_obj/obj_files/maple_example2.obj" #@param {type:"string"}

#@markdown Enter how many samples to generate
test_num_samples = 20    #@param {type:"integer"}
#@markdown Enter export location (folder/directory).   
export_location = f"/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/exportObjects/{id}" #@param {type:"string"}



if id and ckpt_file_location:
    raise Exception("Only one of id / ckpt_file_location can be set!")
if (not id) and (not ckpt_file_location):
    raise Exception("Please set id / ckpt_file_location!")

if id:
    print("id:", id)
    print("project_name:", project_name)
else:
    print("selected_model:", selected_model)
    print("ckpt_file_location:", ckpt_file_location)
print("test_num_samples:", test_num_samples)
print("export_location:", export_location)


In [ ]:
from argparse import Namespace, ArgumentParser
#@markdown Install wandb and log in
entity="bugan"
rev_number = None
if id:
    !pip install wandb
    output.clear()
    #find wandb API key file to auto login
    import wandb
    wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
    wandb_local_netrc_path = Path("/root/.netrc")
    if wandb_drive_netrc_path.exists():
        import shutil

        print("Wandb .netrc file found, will use that to log in.")
        shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
    else:
        print(
            f"Wandb config not found at {wandb_drive_netrc_path}.\n"
            f"Using manual login.\n\n"
            f"To use auto login in the future, finish the manual login first and then run:\n\n"
            f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
            f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
            f"Then that file will be used to login next time.\n"
        )

    !wandb login

    #read information (run config, etc) stored online
        #all config will be replaced by the stored one in wandb
    api = wandb.Api()
    previous_run = api.run(f"{entity}/{project_name}/{id}")
    config = Namespace(**previous_run.config)
        #load selected_model, rev_number in the config
    if hasattr(config, "selected_model"):
        selected_model = config.selected_model
    if hasattr(config, "rev_number"):
        rev_number = config.rev_number

    run = wandb.init(project=project_name, id=id, entity=entity, resume=True, dir="./", group=selected_model)

    output.clear()
    print("run id: " + str(wandb.run.id))
    print("run name: " + str(wandb.run.name))
    print("ok!")

# To just train a model, no edits should be required in any cells below.

In [ ]:
import os
import io
import sys
from pathlib import Path
os.environ["WANDB_MODE"] = "dryrun"

%cd /content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/

run_path = "./"

!apt-get update

!pip install pytorch-lightning
!pip install trimesh
!apt install -y xvfb
!pip install trimesh xvfbwrapper
#bugan package
if rev_number is not None:
    %pip install --upgrade git+https://github.com/buganart/BUGAN.git@{rev_number}#egg=bugan
else:
    %pip install --upgrade git+https://github.com/buganart/BUGAN.git#egg=bugan

import bugan
#EXTRACT package version
    #switch stdout to temperary stringIO
old_stdout = sys.stdout
temp_stdout = io.StringIO()
sys.stdout = temp_stdout
    #get version
%pip freeze | grep bugan
version = temp_stdout.getvalue()
rev_number = version.split("+g")[1].rstrip()
    #switch back stdout
sys.stdout = old_stdout

output.clear()
print("bugan package revision number: " + str(rev_number))
print('ok!')

In [ ]:
import io
from io import BytesIO
import zipfile
import trimesh
import numpy as np


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger


from bugan.functionsPL import *
from bugan.modelsPL import VAEGAN, GAN, VAE_train

# Ignore excessive warnings
import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

from xvfbwrapper import Xvfb

#load model (either VAEGAN / VAE / GAN)

In [ ]:
if id:
    checkpoint_path = os.path.join(wandb.run.dir, 'checkpoint.ckpt')
    load_checkpoint_from_cloud(checkpoint_path = 'checkpoint.ckpt')
else:
    checkpoint_path = ckpt_file_location

#model
if selected_model == "VAEGAN":
    MODEL_CLASS = VAEGAN
elif selected_model == "GAN":
    MODEL_CLASS = GAN
else:
    MODEL_CLASS = VAE_train
    
model = MODEL_CLASS.load_from_checkpoint(checkpoint_path)
model = model.eval().to(device)

#obtain latent vector and store generated mesh files
mesh_file = []
latent_array = []

In [ ]:
#helper function

#for VAEGAN / VAE
def reconstruct_file(vae, f_loc):
    f = Path(f_loc)
    m = trimesh.load(f, force="mesh")
    m = mesh2arrayCentered(m, array_length=32)
    m = m[np.newaxis,np.newaxis,:,:,:]
    m = torch.Tensor(m).float().type_as(vae.vae_encoder.dis_fc1[0].weight)
    m_rec, z, _ = vae(m, output_all=True)

    m_rec = m_rec.detach().cpu().numpy()
    m_rec = m_rec[0,0,:,:,:]
    m_rec_bool_array = m_rec > 0
    voxelmesh = netarray2mesh(m_rec_bool_array)
    return voxelmesh, z[0].detach().cpu().numpy()

#for GAN
def create_mesh_random(generator):
    z = torch.randn(1, generator.z_size).type_as(generator.gen_fc.weight)
    generated_tree = generator(z)[0, 0, :, :, :]
    generated_tree = generated_tree.detach().cpu().numpy()

    mesh_bool_array = generated_tree > 0
    voxelmesh = netarray2mesh(mesh_bool_array)
    return voxelmesh, z[0].detach().cpu().numpy()

#latent space walk (for VAE/VAEGAN)
$Requirement:$

The code below requires **2 meshfiles** to generate 2 latent vectors [$z_1$, $z_2$].

$Description:$

The VAE in the model has 2 components: encoder / decoder.

To generate 2 latent vectors [$z_1$, $z_2$]:

$3D mesh 1 → encoder → latent \ space \ vector \ z1$

$3D mesh 2 → encoder → latent \ space \ vector \ z2$

Then, a line between $z_1$ and $z_2$ will be drawn, and latent space vectors $Z_0, Z_1, ..., Z_{n-1}$ in between will be sampled evenly accroding to the test_num_samples ($n$) set above.

$n = test\_num\_samples, i \in [0, n-1]$

$Z_i = \frac{i}{n-1} z_1 + \frac{n-1-i}{n-1} z_2$ (Note that $Z_0 = z_1$ and $Z_{n-1} = z_2$)

Finally, new meshs will be produced based on those latent space vectors $Z_0, Z_1, ..., Z_{n-1}$:

$latent \ space \ vector \ Z_i → decoder → generated \ mesh \ m_i$

All meshes $m_0, m_1, ..., m_i$ will be stored in the export_location set above.

In [ ]:
if selected_model == "VAEGAN" or selected_model == "VAE":
    vae = model.vae

    file_loc = [meshfile1, meshfile2]
    for f_loc in file_loc:
        _, z = reconstruct_file(vae, f_loc)
        latent_array.append(z)
        
    #latent space walk
    latent_vectors = np.linspace(latent_array[0], latent_array[1], num=test_num_samples)
    # latent_vectors = np.random.randn(test_num_samples,128)
    # print(latent_vectors.shape)

    #genereate meshes from the latent_vectors
    latent_vectors = torch.Tensor(latent_vectors).float().type_as(vae.vae_encoder.dis_fc1[0].weight)
    meshes = vae.vae_decoder(latent_vectors)[:, 0, :, :, :]
    meshes = meshes.detach().cpu().numpy()

    #create export directory
    export_location = Path(export_location)
    print(export_location.exists())
    if not export_location.exists():
        export_location.mkdir(parents=True)

    #store generated meshes to export_location
    for n in range(test_num_samples):
        sample_tree_array = meshes[n] > 0
        voxelmesh = netarray2mesh(sample_tree_array)
        mesh_file.append(voxelmesh)
        export_path = export_location / f"sample_{n}.obj"
        voxelmesh.export(file_obj=export_path, file_type="obj")
    print("ok!")

In [ ]:
# #visualize
# voxelmesh = mesh_file[0]
# #rotate mesh a bit so the color/texture is displayed correctly
# voxelmesh = voxelmesh.apply_transform(trimesh.transformations.rotation_matrix(1e-8, (0,1,0)))
# voxelmesh.show()

#latent space walk (for GAN)
$Requirement:$

The code below requires user to select 2 latent vectors [$z_1$, $z_2$] by manually running the 2 select latent vector cells below (One cell for $z_1$ and one cell for $z_2$). 

Everytime the cell for $z_1$ runs, a new latent vector $z_1$ is generated and replaced the stored latent vector value. The user can check the appearance of the generated mesh from the latent vector $z_1$ to decide whether to keep $z_1$ (same logic for the $z_2$).

Please rerun the cell for $z_1$ and $z_2$ until 2 nice-looking meshes appears.

$Description:$

The GAN mainly use the component generator to generate meshes.

To choose 2 latent vectors [$z_1$, $z_2$]:

$random \ generator → latent \ space \ vector \ $z_1$ → generator → generated \ mesh \ m_1$

$random \ generator → latent \ space \ vector \ $z_2$ → generator → generated \ mesh \ m_2$

Then, a line between $z_1$ and $z_2$ will be drawn, and latent space vectors $Z_0, Z_1, ..., Z_{n-1}$ in between will be sampled evenly accroding to the test_num_samples ($n$) set above.

$n = test\_num\_samples, i \in [0, n-1]$

$Z_i = \frac{i}{n-1} z_1 + \frac{n-1-i}{n-1} z_2$ (Note that $Z_0 = z_1$ and $Z_{n-1} = z_2$)

Finally, new meshs will be produced based on those latent space vectors $Z_0, Z_1, ..., Z_{n-1}$:

$latent \ space \ vector \ Z_i → generator → generated \ mesh \ m_i$

All meshes $m_0, m_1, ..., m_i$ will be stored in the export_location set above.

In [ ]:
if selected_model == "GAN":
    generator = model.generator
    latent_array = np.zeros((2, generator.z_size))
else:
    #terminate script here if selected_model is not GAN. (VAEGAN/GAN part located above)
    raise Exception("selected_model is not GAN. Terminate here!")

In [ ]:
#select latent vector 1
#please keep rerun this cell until "good" mesh appears
voxelmesh = None
voxelmesh, z = create_mesh_random(generator)
latent_array[0] = z

#rotate mesh a bit so the color/texture is displayed correctly
voxelmesh = voxelmesh.apply_transform(trimesh.transformations.rotation_matrix(1e-8, (0,1,0)))
# mesh.show() does not work within if statement?
voxelmesh.show()

In [ ]:
#select latent vector 2
#please keep rerun this cell until "good" mesh appears
voxelmesh = None
voxelmesh, z = create_mesh_random(generator)
latent_array[1] = z

#rotate mesh a bit so the color/texture is displayed correctly
voxelmesh = voxelmesh.apply_transform(trimesh.transformations.rotation_matrix(1e-8, (0,1,0)))
# mesh.show() does not work within if statement?
voxelmesh.show()

In [ ]:
#latent space walk
latent_vectors = np.linspace(latent_array[0], latent_array[1], num=test_num_samples)
# latent_vectors = np.random.randn(test_num_samples,128)
# print(latent_vectors.shape)

#genereate meshes from the latent_vectors
latent_vectors = torch.Tensor(latent_vectors).float().type_as(generator.gen_fc.weight)
meshes = generator(latent_vectors)[:, 0, :, :, :]
meshes = meshes.detach().cpu().numpy()

#create directory
export_location = Path(export_location)
print(export_location.exists())
if not export_location.exists():
    export_location.mkdir(parents=True)

for n in range(test_num_samples):
    sample_tree_array = meshes[n] > 0
    voxelmesh = netarray2mesh(sample_tree_array)
    mesh_file.append(voxelmesh)
    export_path = export_location / f"sample_{n}.obj"
    voxelmesh.export(file_obj=export_path, file_type="obj")
print("ok!")

In [ ]:
# #visualize
# voxelmesh = mesh_file[0]
# #rotate mesh a bit so the color/texture is displayed correctly
# voxelmesh = voxelmesh.apply_transform(trimesh.transformations.rotation_matrix(1e-8, (0,1,0)))
# voxelmesh.show()